# LSTM Stock Price Prediction

Demonstration notebook for the stock prediction workflow.

In [ ]:
import sys
from pathlib import Path

root = Path("..", "src").resolve().parent
if str(root) not in sys.path:
    sys.path.append(str(root))

from src import data_fetch, preprocessing, models, evaluate

raw = data_fetch.load_history(symbol="AAPL", start="2020-01-01", cache_dir=Path("../data"))
features = preprocessing.add_technical_indicators(raw)
supervised = preprocessing.make_supervised(features, target_col="Close", horizon=1)
feature_cols = [c for c in supervised.columns if not c.endswith("target_1")]
target_col = "Close_target_1"
X = supervised[feature_cols].select_dtypes("number").fillna(0.0)
y = supervised[target_col]
split = preprocessing.train_test_split_scaled(X, y)

X_train_seq, y_train_seq = preprocessing.create_lstm_sequences(split.X_train, split.y_train)
X_test_seq, y_test_seq = preprocessing.create_lstm_sequences(split.X_test, split.y_test)

model = models.LSTMPriceModel()
history = model.fit(X_train_seq, y_train_seq)
preds = model.predict(X_test_seq)
report = evaluate.regression_report(y_test_seq, preds)
report.as_dict()

In [ ]:
fig, ax = evaluate.plot_predictions(y_test_seq, preds, title="LSTM Predictions")
fig